## SQLite library in python
1. Book (BookID, Title, Genre)
2. Member (MemberNumber, FamilyName, GivenName)
3. Loan (LoanID, MemberNumber, BookID, DateLoaned, Returned)

In [ ]:
#import library
import sqlite3

#create connection
con = sqlite3.connect('newdb.db')
cursor = con.cursor() #cursor is for retrieving data

#
# create, select statements here.
#

#commit to save changes
con.commit()

#close everything that you created
cursor.close()
con.close()

In [ ]:
#if a question tells you to write a function which does something in SQLite, it's recommended to create a new connection, do something, and close the connection all within one function.

#if you ever get a "database is locked" error, .close() for all your existing connections and cursors, restart notebook, try again.

#### create and select statements

for cheat sheet on how to write statements, go [here](https://www.notion.so/jc-database/Database-SQLite-321b11c40d9e4a7d94ee8f53fad41cb0#d20bcc9c9efd4abba315ad95dce6b4bf) and scroll to the bottom.

In [ ]:
# sample code for creating tables, defining primary key and foreign keys.
con.execute('CREATE TABLE IF NOT EXISTS Book (BookID TEXT PRIMARY KEY, Title TEXT, Genre TEXT)')
con.execute('CREATE TABLE IF NOT EXISTS Member (MemberNumber TEXT PRIMARY KEY, FamilyName TEXT, GivenName TEXT)')
con.execute('CREATE TABLE IF NOT EXISTS Loan (LoanID TEXT PRIMARY KEY, MemberNumber TEXT, BookID TEXT, DateLoaned TEXT, Returned BOOLEAN, FOREIGN KEY (MemberNumber) REFERENCES Member(MemberNumber), FOREIGN KEY (BookID) REFERENCES Book(BookID))')

In [ ]:
# sample code for select statements
# exactly the same as normal tbh
loanID=input()
cursor.execute("""SELECT Book.BookID, Book.Title FROM Book INNER JOIN Loan ON Loan.BookID=Book.BookID WHERE LoanID=? ORDER BY Book.Title ASC""", (loanID,))

result=cursor.fetchall() #returns some kind of cursor object. need to iterate through to get the actual values
for book in result:
    print(book[0],book[1]) # prints bookID,title.

#### adding into database from text file
since this is sample code, I won't write the sqlite3.connect(dbname) part again, but just assume that the connection is active.

In [ ]:
with open('BOOK.txt','r') as book:
    data = book.readlines()
    data = [row.strip('\n').split(',') for row in data] #['new' for 'current' in data]
    print(data)
    #file closes automatically

In [ ]:
for line in data:
    con.execute("INSERT INTO Book(BookID, Title, Genre) VAlUES(?,?,?)", (line[0], line[1], line[2]))
    
# if you insert only one thing,
# "INSERT INTO Table(Column) VALUES(?)", (data,) <- note the comma behind 'data'
# if no comma, (data) is just a value, which may be a string or an int or anything.
# if included comma, (data,) is a tuple. which can now be accessed via indexing. variable_name[0]=data

#### extra: why are f-strings vulnerable?

In [1]:
# real demo of why f strings are vulnerable
# for context, ; denotes the end of one statement, so  -- is used to write a comment.
value1='random_ID, random_title, random_genre); DROP TABLE Book; --"'
value2=''
value3=''
statement=f"INSERT INTO Book(BookID, Title, Genre) VAlUES({value1},{value2},{value3})"
print(statement)

# what you will actually be executing will be:
# INSERT INTO Book(BookID, Title, Genre) VAlUES(random_ID, random_title, random_genre); DROP TABLE Book; --",,)
# bad, cos now your table will be gOne...

# so if f strings are vulnerable, how come the ? method works?
# ? are called placeholders so,
# SQLite will parse your statement without the real values first, so it will understand what the statement structure is.
# if you change the structure afterwards using SQL injection, it will be caught bcos the structure is different.

INSERT INTO Book(BookID, Title, Genre) VAlUES(random_ID, random_title, random_genre); DROP TABLE Book; --",,)
